In [ ]:
source("shared.R")
library(repr)

In [ ]:
library("MASS")
data("OME")
?OME # background on the data
head(OME)

In [ ]:
any(is.na(OME)) # check for missing values
dat = subset(OME, OME != "N/A") # manually remove OME missing values identified with "N/A"
dat$OME = factor(dat$OME)
str(dat)

In [ ]:
options(repr.plot.width=8, repr.plot.height=5)

In [ ]:
plot(dat$Age, dat$Correct / dat$Trials )
plot(dat$OME, dat$Correct / dat$Trials )
plot(dat$Loud, dat$Correct / dat$Trials )
plot(dat$Noise, dat$Correct / dat$Trials )

In [ ]:
ddexp = function(x, mu, tau) {
  0.5*tau*exp(-tau*abs(x-mu)) 
}
curve(ddexp(x, mu=0.0, tau=1.0), from=-5.0, to=5.0, ylab="density", main="Double exponential\ndistribution") # double exponential distribution
curve(dnorm(x, mean=0.0, sd=1.0), from=-5.0, to=5.0, lty=2, add=TRUE) # normal distribution
legend("topright", legend=c("double exponential", "normal"), lty=c(1,2), bty="n")

In [ ]:
mod_glm = glm(Correct/Trials ~ Age + OME + Loud + Noise, data=dat, weights=Trials, family="binomial")
summary(mod_glm)

In [ ]:
coefficients(mod_glm)

In [ ]:
anova(mod_glm)

In [ ]:
X = model.matrix(mod_glm)[,-1] # -1 removes the column of 1s for the intercept
head(X)

In [ ]:
mod_string = " model {
for (i in 1:length(y)) {
    y[i] ~ dbin(phi[i], n[i])
    logit(phi[i]) = b0 + b[1]*Age[i] + b[2]*OMElow[i] + b[3]*Loud[i] + b[4]*Noiseincoherent[i]
    }
    
    b0 ~ dnorm(0.0, 1.0/5.0^2)
    for (j in 1:4) {
        b[j] ~ dnorm(0.0, 1.0/4.0^2)
    }    
} "
inits = function() {
    inits = list("b" = rnorm(4, 0, 16))
}
data_jags = as.list(as.data.frame(X))
data_jags$y = dat$Correct # this will not work if there are missing values in dat (because they would be ignored by model.matrix). Always make sure that the data are accurately pre-processed for JAGS.
data_jags$n = dat$Trials
str(data_jags) # make sure that all variables have the same number of observations (712).
#jags_glm = jags_model_compile(mod_string, data_jags, burnout = 1e3)
jags_glm = jags.model(textConnection(mod_string), data = data_jags, n.chains = 3)
update(jags_glm, 1e3)
#plot(residuals(mod_glm, type="deviance"))
#plot(fitted(mod_glm), dat$Correct/dat$Trials)

In [ ]:
jags_glm_sim = jags_model_sample(mod_ = jags_glm,  var_names_ = c("b0", "b"), n_iter_ = 5e3)

In [ ]:
jags_glm_diag = jags_model_diag(samples_ = jags_glm_sim$samples)

In [ ]:
head(jags_glm_sim$samples_cmb[,1:4])

In [ ]:
par(mfrow=c(3,2))
densplot(jags_glm_sim$samples_cmb[,1:5])

In [ ]:
col_means = colMeans(jags_glm_sim$samples_cmb[,1:5])

In [ ]:
col_means['b0']

In [ ]:
y_est = c(60, 0, 50, 1, 1)

In [ ]:
1/ ( 1 + exp( - t(col_means) %*% y_est))

In [ ]:
cbind(head(X), "const" = 1)

In [ ]:
install.packages("gtools")
library("gtools")

In [ ]:
Z = cbind(X, "c" = 1)

In [ ]:
?inv.logit

In [ ]:
phat = inv.logit(Z%*%col_means)

In [ ]:
head(phat)

In [ ]:
(tab0.7 = table(phat > 0.7, (dat$Correct / dat$Trials) > 0.7))
sum(diag(tab0.7)) / sum(tab0.7)

In [ ]:
dist(phat)